
##  Medallion Architecture (Multi-hop)


- Medallion Architecture is a **layered approach** for building reliable, scalable data lakes:

| Layer                 | Description                                                          | Data Quality           | Purpose                                                        |
| --------------------- | -------------------------------------------------------------------- | ---------------------- | -------------------------------------------------------------- |
| **Bronze** (Raw)      | Raw ingestion from source systems (e.g., logs, events, files)        | Raw data, may be dirty | Capture original data for audit and backup                     |
| **Silver** (Cleansed) | Cleaned and filtered data, duplicates removed, basic transformations | Cleaned, enriched      | Enable business logic, filtering, and filtering of bad records |
| **Gold** (Aggregated) | Aggregated, business-level data ready for analytics and BI           | Curated, optimized     | Power dashboards, ML, and reporting                            |

- **Multi-hop** refers to data pipelines where data flows through multiple stages or tables before reaching the final presentation layer.
- Data flows through these layers progressively, increasing in quality and usability.



#### Silver Layer (Parse and Flatten)

- Parse the JSON string column order_details with from_json() and a defined schema
- Explode the nested array products so each product is one row
- Flatten nested structs like delivery address and payment info
- Cast columns to proper types (int, float, date, etc.)
- Save as Delta table `orders_silver`

#### Gold Layer (Business Aggregations)

- Aggregate to get metrics like total sales and order count per product category & payment method
- Save as Gold Delta table